*   **Año:** 2024
*   **Alumno/a:** Gonzalo Degiuseppe
*   **Legajo:** 47.126.858

# Pydantic
Pydantic es una libreria rapida y extensible que nos permite validar datos usando tipos de datos de Python.


Primero, importe `pydantic`.

In [2]:
import pydantic as pyd

Supongamos que tenemos una lista de clientes (llamemoslo en el codigo `Client`). Los clientes tienen dos campos: DNI (un entero) y nacionalidad (un `string`). Cree el modelo base de `Client` (en forma de clases de Python).

In [3]:
class Client(pyd.BaseModel):
    dni: pyd.conint(strict=True)
    nacionalidad: str

Cree a un usuario con documento 39.755.010 y nacionalidad 'Argentina'. Muestre todos sus campos.

In [4]:
try:
    cliente = Client(dni = 39755010, nacionalidad = "Argentina")
    print("Cliente:")
    print(cliente)
except pyd.ValidationError as e:
    print(e)

Cliente:
dni=39755010 nacionalidad='Argentina'


Intente crear al usuario con un documento en forma de `string`. Deberia fallar...

In [5]:
try:
    cliente_fallo = Client(dni = "39755010", nacionalidad = "Argentina")
    print("Cliente inválido:")
    print(cliente_fallo)
except pyd.ValidationError as e:
    print("Error en la validación del cliente:")
    print(e)

Error en la validación del cliente:
1 validation error for Client
dni
  Input should be a valid integer [type=int_type, input_value='39755010', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_type


Hemos detectado que ciertos clientes tienen nacionalidades que no existen. Ademas, hay numeros de documento negativos y se tiene que poder agregar la fecha de registro de los clientes (que no pueden ser del futuro). Cambiar la definicion del cliente para que estas cosas no ocurran. Despues de la siguiente celda, cree otras 3 mas probando un caso donde deberia funcionar y otros dos en los que no.

In [6]:
from datetime import date
from typing import Literal

class Client(pyd.BaseModel):
    dni: pyd.conint(gt=0)
    nacionalidad: Literal["Argentina", "Uruguay", "Brasil"]
    fecha_registro: date

    @pyd.validator("fecha_registro")
    def check_fecha_registro(cls, v):
        if v > date.today():
            raise ValueError("La fecha de registro no puede ser del futuro")
        return v

<ipython-input-6-21506db8457c>:9: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @pyd.validator("fecha_registro")


In [7]:
try:
    cliente = Client(dni = 39755010, nacionalidad = "Argentina", fecha_registro = date(2024, 10, 18))
    print("Cliente:")
    print(cliente)
except pyd.ValidationError as e:
    print("Error")
    print(e)

Cliente:
dni=39755010 nacionalidad='Argentina' fecha_registro=datetime.date(2024, 10, 18)


In [8]:
try:
    cliente_fallo = Client(dni = -12345678, nacionalidad = "Uruguay", fecha_registro = date(2024, 10, 18))
    print("Cliente:")
    print(cliente_fallo)
except pyd.ValidationError as e:
    print("Error")
    print(e)

Error
1 validation error for Client
dni
  Input should be greater than 0 [type=greater_than, input_value=-12345678, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than


In [9]:
try:
    ciente_fallo2 = Client(dni = 12345678, nacionalidad = "kfnjsnbgi", fecha_registro = date(2024, 10, 18))
    print("Cliente:")
    print(cliente_fallo2)
except pyd.ValidationError as e:
    print("Error")
    print(e)

Error
1 validation error for Client
nacionalidad
  Input should be 'Argentina', 'Uruguay' or 'Brasil' [type=literal_error, input_value='kfnjsnbgi', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/literal_error


# Pandera
Hacer lo que se pidio en la ultima celda anterior, pero con Pandera.

In [10]:
!pip install pandera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.9 MB/s eta 0:00:00


In [18]:
import pandera as pa
from datetime import date
import pandas as pd

schema = pa.DataFrameSchema(
    {
        "dni": pa.Column(int, checks=pa.Check.ge(1)),
        "nacionalidad": pa.Column(str, checks=pa.Check.isin(["Argentina", "Uruguay", "Brasil"])),
        "fecha_registro": pa.Column(
            pa.DateTime,
            checks=pa.Check(
                lambda fecha: fecha <= pd.Timestamp(date.today()),
                element_wise=True
            )
        ),
    }
)

In [21]:
df = pd.DataFrame({
    "dni": [12345678],
    "nacionalidad": ["Argentina"],
    "fecha_registro": [pd.Timestamp("2024-10-18")]
})

try:
    schema.validate(df)
    print("El DataFrame es válido")
except pa.errors.SchemaError as e:
    print(f"Error de validación: {e}")

El DataFrame es válido


In [23]:
df2 = pd.DataFrame({
    "dni": [-23456789],
    "nacionalidad": ["Uruguay"],
    "fecha_registro": [pd.Timestamp("2024-10-18")]
})


try:
    schema.validate(df2)
    print("El dataframe es válido")
except pa.errors.SchemaError as e:
    print(f"Error de validación: {e}")

Error de validación: Column 'dni' failed element-wise validator number 0: greater_than_or_equal_to(1) failure cases: -23456789


In [24]:
df3 = pd.DataFrame({
    "dni": [12345678],
    "nacionalidad": ["Peru"],
    "fecha_registro": [pd.Timestamp("2024-10-18")]
})

try:
    schema.validate(df3)
    print("El dataframe es válido")
except pa.errors.SchemaError as e:
    print(f"Error de validación: {e}")

Error de validación: Column 'nacionalidad' failed element-wise validator number 0: isin(['Argentina', 'Uruguay', 'Brasil']) failure cases: Peru
